# Search Motifs by scanning 


## Scanning-and-Scoring a Motif
Given a possible motif, we find its best match in each sequence.

In [1]:
def ScanAndScoreMotif(DNA, motif):
    """ Given a list of sequences in DNA and a motif,
    find the best alignments of the motif in all sequences
    and compute the total Hamming distances"""
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in range(len(seq)-k+1):
            HammingDist = sum([1 for i in range(k) if motif[i] != seq[s+i]])
            if (HammingDist < minHammingDist):
                bestS = s
                minHammingDist = HammingDist
        bestAlignment.append(bestS)
        totalDist += minHammingDist
    return bestAlignment, totalDist

In [2]:
seqApprox = [
    'tagtggtcttttgagtgtagatctgaagggaaagtatttccaccagttcggggtcacccagcagggcagggtgacttaat',
    'cgcgactcggcgctcacagttatcgcacgtttagaccaaaacggagttggatccgaaactggagtttaatcggagtcctt',
    'gttacttgtgagcctggttagacccgaaatataattgttggctgcatagcggagctgacatacgagtaggggaaatgcgt',
    'aacatcaggctttgattaaacaatttaagcacgtaaatccgaattgacctgatgacaatacggaacatgccggctccggg',
    'accaccggataggctgcttattaggtccaaaaggtagtatcgtaataatggctcagccatgtcaatgtgcggcattccac',
    'tagattcgaatcgatcgtgtttctccctctgtgggttaacgaggggtccgaccttgctcgcatgtgccgaacttgtaccc',
    'gaaatggttcggtgcgatatcaggccgttctcttaacttggcggtgcagatccgaacgtctctggaggggtcgtgcgcta',
    'atgtatactagacattctaacgctcgcttattggcggagaccatttgctccactacaagaggctactgtgtagatccgta',
    'ttcttacacccttctttagatccaaacctgttggcgccatcttcttttcgagtccttgtacctccatttgctctgatgac',
    'ctacctatgtaaaacaacatctactaacgtagtccggtctttcctgatctgccctaacctacaggtcgatccgaaattcg']

%time (bestAlignment, totalDist) = ScanAndScoreMotif(seqApprox, "tagatccgaa")
print(bestAlignment, totalDist)
k = len("tagatccgaa")
for i in range(len(bestAlignment)):
        print(seqApprox[i][bestAlignment[i]:bestAlignment[i]+k])

Wall time: 4 ms
[17, 47, 18, 33, 21, 0, 46, 70, 16, 65] 11
tagatctgaa
tggatccgaa
tagacccgaa
taaatccgaa
taggtccaaa
tagattcgaa
cagatccgaa
tagatccgta
tagatccaaa
tcgatccgaa
Wall time: 2 ms
[17, 47, 18, 33, 21, 0, 46, 70, 16, 65] 11
tagatctgaa
tggatccgaa
tagacccgaa
taaatccgaa
taggtccaaa
tagattcgaa
cagatccgaa
tagatccgta
tagatccaaa
tcgatccgaa


## Scan all possible k-mer motifs to find the best one
How many motifs are to be scanned?

In [3]:
import itertools

def MedianStringMotifSearch(DNA,k):
    """ Consider all possible 4**k motifs"""
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for pattern in itertools.product('acgt', repeat=k):
        motif = ''.join(pattern)
        align, dist = ScanAndScoreMotif(DNA, motif)
        if (dist < minHammingDist):
            bestAlignment = [p for p in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

%time MedianStringMotifSearch(seqApprox,8)

Wall time: 1min 55s


([19, 49, 20, 35, 23, 2, 48, 72, 18, 67], 8, 'gatccgaa')

## Let's consider only Motifs seen in the DNA
Are we guaranteed to get the best motif?

In [5]:
def ContainedMotifSearch(DNA,k):
    """ Consider only motifs from the given DNA sequences"""
    motifSet = set()
    for seq in DNA:
        for i in range(len(seq)-k+1):
            motifSet.add(seq[i:i+k])
    print("%d Motifs in our set" % len(motifSet))
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for motif in motifSet:
        align, dist = ScanAndScoreMotif(DNA, motif)
        if (dist < minHammingDist):
            bestAlignment = [s for s in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

k = 8
%time bestAlignment, minDist, kmer = ContainedMotifSearch(seqApprox, k)
print(bestAlignment, minDist, kmer)
for i in range(len(bestAlignment)):
        print(seqApprox[i][bestAlignment[i]:bestAlignment[i]+k])

718 Motifs in our set
Wall time: 1.34 s
[19, 49, 20, 35, 23, 2, 48, 72, 18, 67] 8 gatccgaa
gatctgaa
gatccgaa
gacccgaa
aatccgaa
ggtccaaa
gattcgaa
gatccgaa
gatccgta
gatccaaa
gatccgaa


## Contained Consensus Motif Search
Note if we only consider kmers in the DNA, the best kmer isn't necessarily the consensus of the alignment. 

The following functions gets the consensus motif and profile given an alignment. It also computes the likelihood of k-mer given a profile. 

In [10]:
import numpy as np

def Consensus(s, DNA, k):
    """ compute the consensus k-Motif of an alignment given offsets into each DNA string.
            s = list of starting indices, 1-based, 0 means ignore, DNA = list of nucleotide strings,
            k = Target Motif length """
    consensus = ''
    for i in range(k):
        # loop over string positions
        cnt = dict(zip("acgt",(0,0,0,0)))
        for j, sval in enumerate(s):
            # loop over DNA strands
            base = DNA[j][sval+i] 
            cnt[base] += 1
        consensus += max(cnt.items(), key=lambda tup: tup[1])[0]
    return consensus


def ContainedConsensusMotifSearch(DNA,k):
    bestAlignment, minHammingDist, kmer = ContainedMotifSearch(DNA,k)
    motif = Consensus(bestAlignment,DNA,k)
    newAlignment, HammingDist = ScanAndScoreMotif(DNA, motif)
    return newAlignment, HammingDist, motif

%time newAlignment, HammingDist, motif = ContainedConsensusMotifSearch(seqApprox,10)
print(motif)
print(newAlignment, HammingDist)

709 Motifs in our set
Wall time: 1.3 s
tagatccgaa
[17, 47, 18, 33, 21, 0, 46, 70, 16, 65] 11


## Motif Profiles

Represent a motif as a profile of base frequencies in each column. 

In [27]:
def Profile(s, DNA, k):
    """ Compute the profile of k-mer motifs in a DNA string 
        s = list of starting positions, 
        DNA = list of nucleotide strings,
        k = Target Motif length"""
    profile = np.zeros((4, k))
    for i in range(k):
        # loop over string positions
        cnt = dict(zip("acgt",(0,0,0,0)))
        for j, sval in enumerate(s):
            # loop over DNA strands
            base = DNA[j][sval+i] 
            cnt[base] += 1
        profile[:, i] = np.fromiter(cnt.values(), dtype=float) / (float)(len(s))
    return profile

def LikelihoodKmerProfile(kmer, profile):
    """Given a profile, compute the probability of a kmer string"""
    # check that kmer length matches with profile length
    assert len(kmer) == profile.shape[1]
    prob = 1.0
    idx = dict(zip("acgt", (0, 1, 2, 3)))
    for i in range(len(kmer)):
        prob = prob * profile[idx[kmer[i]], i]
    return prob

def ScanAndScoreProfile(DNA, profile):
    """ Given a list of sequences in DNA and a profile,
    find the best alignments of the profile in all sequences
    return:
        - the best alignment for this profile
        - the total likelihood of the best alignment
    """
    bestAlignment = []
    totalLikelihood = 1
    k = profile.shape[1] 
    for seq in DNA:
        maxProbProfile = 0
        for s in range(len(seq)-k+1):
            prob = LikelihoodKmerProfile(seq[s:s+k], profile)
            if (prob > maxProbProfile):
                bestS = s
                maxProbProfile = prob
        bestAlignment.append(bestS)
        totalLikelihood *= maxProbProfile
    return bestAlignment, totalLikelihood

In [28]:
alignment = [10, 12, 11, 3, 0, 5, 8, 56, 46, 32]
profile = Profile(alignment, seqApprox, 10)
print(profile)
motif = "tccaggctat"
print(LikelihoodKmerProfile(motif, profile))
bestProfileAlignment = ScanAndScoreProfile(seqApprox, profile)
print(bestProfileAlignment)

[[0.3 0.1 0.  0.6 0.2 0.1 0.  0.  0.6 0.1]
 [0.1 0.5 0.6 0.  0.2 0.1 0.5 0.1 0.1 0.1]
 [0.1 0.  0.4 0.3 0.5 0.5 0.3 0.3 0.  0.2]
 [0.5 0.4 0.  0.1 0.1 0.3 0.2 0.6 0.3 0.6]]
0.00243
([10, 21, 37, 3, 0, 5, 8, 56, 47, 32], 8.671917315939523e-42)


## Randomized Motif Search
Searches for a k-length motif profile in given DNA sequences that maximize the total likelihood. It begins with a motif profile derived from a set of random start positions in the sequence. It iteratively update the start positions and profile until it converges. Try multiple random intializations to get the best result.

In [58]:
import numpy as np
from numpy.random import randint
from numpy.random import choice

def RandomizedMotifSearch(DNA, k, r = 30):
    """ Searches for a k-length motif that appears 
    in all given DNA sequences. It begins with a
    random set of candidate consensus motifs 
    derived from the data. It refines the motif
    using Gibbs sampler until a true consensus emerges.
    DNA - list of DNA sequences
    k - length of kmer motif
    r - number of random initial alignments
    """
    bestAlignment = []
    bestLikelihood = 0
    for sample in range(r):    # Try r different random initial alignments
        #print("sample = ", sample)
        # Start with a random alignment
        randomAlignment = [randint(len(DNA[j])-k+1) for j in range(len(DNA))]
        # Get the profile corresponding to this alignment
        converged = False;
        while not converged:
            profile = Profile(randomAlignment, DNA, k)
            # Get the best alignment corresponding to this profile
            newAlignment, likelihood = ScanAndScoreProfile(DNA, profile)
            if likelihood > bestLikelihood :
                bestAlignment = newAlignment.copy()
                #print(bestLikelihood, " changed to ", likelihood)
                bestLikelihood = likelihood
                
            if newAlignment == randomAlignment:   # stop when converged
                converged = True
            else:
                randomAlignment = newAlignment    # repeat with the updated alignment
        
    return bestAlignment, bestLikelihood, Profile(bestAlignment, DNA, k)
    

## Let's try it

In [64]:
%time bestAlignment, bestLikelihood, profile = RandomizedMotifSearch(seqApprox,10, r = 500)
print(profile, bestLikelihood)
print(bestAlignment)
print(Consensus(bestAlignment, seqApprox, 10))

Wall time: 11.4 s
[[0.  0.8 0.1 0.9 0.  0.  0.  0.2 0.9 1. ]
 [0.1 0.1 0.  0.  0.1 0.9 0.9 0.  0.  0. ]
 [0.  0.1 0.9 0.1 0.  0.  0.  0.8 0.  0. ]
 [0.9 0.  0.  0.  0.9 0.1 0.1 0.  0.1 0. ]] 1.4749519686370307e-15
[17, 47, 18, 33, 21, 0, 46, 70, 16, 65]
tagatccgaa


In [61]:
for i in range(10):
    bestAlignment, bestLikelihood, profile = RandomizedMotifSearch(seqApprox,10)
    print(profile, bestLikelihood)
    print(Consensus(bestAlignment, seqApprox, 10))

[[0.  0.7 0.  0.8 0.2 0.1 0.  0.  0.6 0.9]
 [0.1 0.  0.1 0.  0.3 0.7 0.7 0.1 0.1 0. ]
 [0.  0.2 0.8 0.  0.  0.  0.  0.9 0.  0. ]
 [0.9 0.1 0.1 0.2 0.5 0.2 0.3 0.  0.3 0.1]] 6.716378594019652e-28
tagatccgaa
[[0.1 0.7 0.1 0.8 0.  0.  0.  0.2 0.8 0.9]
 [0.  0.1 0.  0.  0.1 0.8 0.9 0.  0.  0. ]
 [0.  0.2 0.9 0.2 0.1 0.  0.  0.8 0.  0.1]
 [0.9 0.  0.  0.  0.8 0.2 0.1 0.  0.2 0. ]] 2.525317894852944e-21
tagatccgaa
[[0.1 0.5 0.3 0.8 0.2 0.  0.1 0.  0.8 1. ]
 [0.2 0.1 0.  0.1 0.  0.6 0.8 0.1 0.  0. ]
 [0.  0.3 0.7 0.1 0.  0.1 0.  0.7 0.  0. ]
 [0.7 0.1 0.  0.  0.8 0.3 0.1 0.2 0.2 0. ]] 3.2510766269377904e-29
tagatccgaa
[[0.  1.  0.1 0.7 0.  0.  0.  0.4 0.7 0.9]
 [0.3 0.  0.1 0.  0.1 0.8 0.7 0.  0.  0.1]
 [0.  0.  0.8 0.3 0.  0.  0.1 0.6 0.1 0. ]
 [0.7 0.  0.  0.  0.9 0.2 0.2 0.  0.2 0. ]] 1.0829416377001932e-23
tagatccgaa
[[0.2 0.2 0.1 0.  0.5 0.2 1.  0.1 0.1 0.1]
 [0.  0.  0.2 0.5 0.2 0.  0.  0.4 0.8 0.2]
 [0.7 0.  0.6 0.  0.  0.8 0.  0.  0.1 0. ]
 [0.1 0.8 0.1 0.5 0.3 0.  0.  0.5 0.  0.7]] 4

In [ ]:
tagatccgaa